In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --upgrade datasets sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


# Prepare data

In [4]:
import time
import torch
import torch.nn as nn
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_df = pd.read_table('/content/drive/MyDrive/NLP_Final_Project/data/train.tsv')
val_df = pd.read_table('/content/drive/MyDrive/NLP_Final_Project/data/dev.tsv')
test_df = pd.read_table('/content/drive/MyDrive/NLP_Final_Project/data/test.tsv')

device

device(type='cuda')

In [5]:
test_class_0 = test_df[test_df['class'] == 0]
test_class_1 = test_df[test_df['class'] == 1]

sampled_test_class_0 = test_class_0.sample(n=600, random_state=42)
sampled_test_class_1 = test_class_1.sample(n=300, random_state=42)

sampled_test_data = pd.concat([sampled_test_class_0, sampled_test_class_1])

test_df = test_df.drop(sampled_test_data.index)
train_df = pd.concat([train_df, sampled_test_data])

# Load tokenizer and model for TinyBERT, BERT

In [6]:
tinybert_tokenizer = BertTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D')
tinybert_model = BertForSequenceClassification.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', num_labels=2).to(device)
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bertfull_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

trainds=Dataset.from_pandas(train_df.rename(columns={"class":"labels"}))
evalds=Dataset.from_pandas(val_df.rename(columns={"class":"labels"}))
testds=Dataset.from_pandas(test_df.rename(columns={"class":"labels"}))

def tinybert_tokenization(batch):
    return tinybert_tokenizer(batch["sentence"],padding=True,truncation=True,return_tensors="pt").to(device)
def bertfull_tokenization(batch):
    return bert_tokenizer(batch["sentence"],padding=True,truncation=True,return_tensors="pt").to(device)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train TinyBERT

In [15]:
tinybert_training_args=TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP_Final_Project/tinybert_ckpt",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    learning_rate=1e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

In [16]:
tb_encoded_train=trainds.map(tinybert_tokenization ,batched=True,batch_size=None)
tb_encoded_eval=evalds.map(tinybert_tokenization,batched=True,batch_size=None)
tb_encoded_test=testds.map(tinybert_tokenization,batched=True,batch_size=None)
trainer=Trainer(
    model=tinybert_model,
    args=tinybert_training_args,
    train_dataset=tb_encoded_train,
    eval_dataset=tb_encoded_eval,
    compute_metrics=compute_metrics
)
trainer.train()

Map:   0%|          | 0/7820 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.343547,0.853211,0.854078,0.853211,0.853196
2,No log,0.328280,0.869266,0.870644,0.869266,0.869054
3,0.294500,0.334322,0.869266,0.871344,0.869266,0.868973
4,0.294500,0.324425,0.878440,0.878995,0.878440,0.878342
5,0.254800,0.326182,0.872706,0.873625,0.872706,0.872556


TrainOutput(global_step=1225, training_loss=0.2675840323311942, metrics={'train_runtime': 61.6004, 'train_samples_per_second': 634.736, 'train_steps_per_second': 19.886, 'total_flos': 87602305056000.0, 'train_loss': 0.2675840323311942, 'epoch': 5.0})

# Train BERT

In [17]:
bert_training_args=TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP_Final_Project/bert_ckpt",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    learning_rate=1e-6,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

In [18]:
bf_encoded_train=trainds.map(bertfull_tokenization ,batched=True,batch_size=None)
bf_encoded_eval=evalds.map(bertfull_tokenization,batched=True,batch_size=None)
bf_encoded_test=testds.map(bertfull_tokenization,batched=True,batch_size=None)
trainer2=Trainer(
    model=bertfull_model,
    args=bert_training_args,
    train_dataset=bf_encoded_train,
    eval_dataset=bf_encoded_eval,
    compute_metrics=compute_metrics
)
trainer2.train()

Map:   0%|          | 0/7820 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.287489,0.907110,0.907249,0.907110,0.907083
2,No log,0.312295,0.907110,0.907144,0.907110,0.907097
3,0.093600,0.313867,0.909404,0.909623,0.909404,0.909368
4,0.093600,0.313830,0.909404,0.909832,0.909404,0.909348
5,0.119700,0.311206,0.905963,0.906016,0.905963,0.905947


TrainOutput(global_step=1225, training_loss=0.10697513191067443, metrics={'train_runtime': 569.7586, 'train_samples_per_second': 68.626, 'train_steps_per_second': 2.15, 'total_flos': 1607444103840000.0, 'train_loss': 0.10697513191067443, 'epoch': 5.0})